In [ ]:
from joblib import dump, load
import spacy
from spacy.lang.ru.examples import sentences
from pprint import pprint
import warnings
warnings.simplefilter('ignore')

from config import CONFIG

In [2]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download ru_core_news_lg

In [3]:
binary_vectorizer = load(f'{CONFIG.MODELS_FOLDER}/{CONFIG.BINARY_VECTORIZER_NAME}') 
binary_classifier = load(f'{CONFIG.MODELS_FOLDER}/{CONFIG.BINARY_CLASSIFER_NAME}') 
multi_vectorizer = load(f'{CONFIG.MODELS_FOLDER}/{CONFIG.MULTI_VECTORIZER_NAME}') 
multi_classifier = load(f'{CONFIG.MODELS_FOLDER}/{CONFIG.MULTI_CLASSIFIER_NAME}') 
nlp = spacy.load("ru_core_news_lg")
categories = ['position_name', 'education_list', 'skills', 'aboutme_info', 'salary', 'experience_list', 'other']

In [4]:
# случайные примеры

sample_1 = """Мужчина
46 лет
Санкт-Петербург
Программист-разработчик
200 000 руб.
Информационные технологии, интернет, телеком
DevOps Linux Nginx Zabbix Python PostgreSQL FreeBSD MySQL Virtualization PHP Unix Bash Git Ansible Jenkins Networking AWS Amazon Web Services Web VPN Docker Apache HTTP Server DNS Unix Shell Scripts CI/CD Atlassian Jira Atlassian Confluence Постановка задач разработчикам разработчикам Product Management Управление разработкой разработкой Data Analysis
Имеется собственный автомобиль. Права категории D
Высшее образование
Быстро осваиваю новую информацию. За неделю вхожу в курс дела и начинаю работать. Увлечения - физика, электроника, компьютеры Занимаюсь пайкой, монтажом радиоэлектронных компонентов(РЭА) Интересуюсь низкоуровневым программирование(Assembler x86 intel , C ) . Английский на уровне Intermediate Характер - спокойный рассудительный
1997. Восточно-Сибирский институт экономики и права, Иркутск. Экономический, Экономист
Июнь 2020 — по настоящее время. Сбербанк. Инженер-программист. Снизил использование трафика облачного сервиса по хранению картинок cloudinary на 38%. Путем внедрения автоматической оптимизации изображений.
"""
sample_2 = """
программист РЭАиП
по договоренности
Уверенный пользователь пк, ремонт пк. С++/C, Java.
Москва
42 года
мужчина
Москва
Россия
Холост , Детей нет
Велотуризм, музыка. Я ответственно подхожу к работе. Выдерживаю дедлайны, даже при условии, что сдать нужно вчера. Коммуникабельный и открытый человек.
ООО "Северсталь". Москва. 2016, январь — 2021, февраль, аккаунт-менеджер. Сборка, настройка, ремонт средств вычислительной техники (компьютеры, мониторы)
2008 г.в.. Среднее профессиональное. МГТУ. ЭВМ и компьютерные сети
"""
samples = (sample_1, sample_2)
for sample in samples:
    print(sample)

Мужчина
46 лет
Санкт-Петербург
Программист-разработчик
200 000 руб.
Информационные технологии, интернет, телеком
DevOps Linux Nginx Zabbix Python PostgreSQL FreeBSD MySQL Virtualization PHP Unix Bash Git Ansible Jenkins Networking AWS Amazon Web Services Web VPN Docker Apache HTTP Server DNS Unix Shell Scripts CI/CD Atlassian Jira Atlassian Confluence Постановка задач разработчикам разработчикам Product Management Управление разработкой разработкой Data Analysis
Имеется собственный автомобиль. Права категории D
Высшее образование
Быстро осваиваю новую информацию. За неделю вхожу в курс дела и начинаю работать. Увлечения - физика, электроника, компьютеры Занимаюсь пайкой, монтажом радиоэлектронных компонентов(РЭА) Интересуюсь низкоуровневым программирование(Assembler x86 intel , C ) . Английский на уровне Intermediate Характер - спокойный рассудительный
1997. Восточно-Сибирский институт экономики и права, Иркутск. Экономический, Экономист
Июнь 2020 — по настоящее время. Сбербанк. Инжене

In [5]:
def extract_entity_from_text(text, entity_label='ORG'):
    doc = nlp(text)
    return [entity for entity in doc.ents if entity.label_ == entity_label]

def extract_entity_from_cv(cv_json):
    if 'education_list' in cv_json:
        cv_json['education_places'] = extract_entity_from_text(cv_json['education_list'])
    if 'experience_list' in cv_json:
        cv_json['experience_places'] = extract_entity_from_text(cv_json['experience_list'])
    return cv_json
        
def parse_cv(cv, binary_vectorizer=binary_vectorizer, binary_classifier=binary_classifier,
             multi_vectorizer=multi_vectorizer, multi_classifier=multi_classifier, nlp=nlp):
    result = {category: '' for category in categories}
    cv_transformed = binary_vectorizer.transform([cv])
    if binary_classifier.predict(cv_transformed)[0] != 1:
        result['is_cv'] = False
        return result
    result['is_cv'] = True
    lines = cv.split('\n')
    for line in lines:
        line_transformed = multi_vectorizer.transform([line])
        category_class = multi_classifier.predict(line_transformed)[0]
        if result[categories[category_class]]:
            result[categories[category_class]] += ' ' + line
        else:
            result[categories[category_class]] = line
        
    result = extract_entity_from_cv(result)
    return result
    
for sample in samples:
    pprint(parse_cv(sample))
    print('\n')

{'aboutme_info': 'Имеется собственный автомобиль. Права категории D Быстро '
                 'осваиваю новую информацию. За неделю вхожу в курс дела и '
                 'начинаю работать. Увлечения - физика, электроника, '
                 'компьютеры Занимаюсь пайкой, монтажом радиоэлектронных '
                 'компонентов(РЭА) Интересуюсь низкоуровневым '
                 'программирование(Assembler x86 intel , C ) . Английский на '
                 'уровне Intermediate Характер - спокойный рассудительный',
 'education_list': '1997. Восточно-Сибирский институт экономики и права, '
                   'Иркутск. Экономический, Экономист',
 'education_places': [Восточно-Сибирский институт экономики и права],
 'experience_list': 'Июнь 2020 — по настоящее время. Сбербанк. '
                    'Инженер-программист. Снизил использование трафика '
                    'облачного сервиса по хранению картинок cloudinary на 38%. '
                    'Путем внедрения автоматической оптимизац